In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 합성곱 연산 (입력 채널 수: 3, 출력 채널 수: 6, 필터 크기: 5x5, stride=1(default))
        self.pool1 = nn.MaxPool2d(2, 2)  # 합성곱 연산 (필터크기 2x2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)  # 합성곱 연산 (입력 채널 수: 6, 출력 채널수: 16, 필터 크기: 5x5, stride=1(default))
        self.pool2 = nn.MaxPool2d(2, 2)  # 합성곱 연산 (필터크기 2x2, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 피쳐맵 16개를 일렬로 피면 16*5*5개의 노드가 생성됨.
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        # conv1 -> relu -> pool1 -> conv2 -> relu -> pool2 -> linear
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))

        x = x.view(-1, 16 * 5 * 5)  # 5x5 피쳐맵 16개를 일렬로 만든다.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x

In [3]:
# transforms.Compose => 모든 변환 함수들을 하나로 조합하는 함수
# 이 함수를 dataloader에 넘기면 이미지 변환 작업이 간단하게 완료된다
transform = transforms.Compose(
    # ToTensor => torch.Tensor로 변환
    [transforms.ToTensor(),
     # Normalize(mean, std, inplace=False) => tensor의 데이터 수치(또는 범위)를 정규화
     # 데이터가 color -> 데이터(3차원(channel*width*height), 0.5는 임의의 값)
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
    )
# datasets => Pytorch가 공식적으로 다운로드 및 사용을 지원하는 데이터셋(CIFAR10 : 클래스 10개의 이미지 가지는 데이터, 3d tensor로 구성)
# root : 경로, train : true면 trainset에서 데이터셋 생성 false면 testset에서 생성
# download : true인 경우 다운로드하고 root 디렉토리에 넣음
# transform : PIL 이미지 가져와 변환된 버전 반환하는 함수(위에서 선언한 transform 적용)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# DataLoader로 배치 형태로 만든다, 배치당 샘플 수=>8, 모든 epoch에서 데이터 섞기
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

Files already downloaded and verified
Files already downloaded and verified
cuda is available


In [4]:
net = Net().to(device) # 모델 선언
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)


In [5]:
# 분류문제이기 때문에 손실함수를 크로스 엔트로피 사용
criterion = nn.CrossEntropyLoss()
# 최적화 방법 -> 모멘텀 활용 (SGD : 확률적 경사 하강법을 구현(선택적으로 모멘텀 포함))
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

loss_ = [] # loss 저장용 리스트
n = len(trainloader) # 배치개수
for epoch in range(10): # 10회 반복
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device) # 배치 데이터
        optimizer.zero_grad() # 배치마다 optimizer 초기화
        outputs = net(inputs) # 노드 10개짜리 예측값 산출
        loss = criterion(outputs, labels) # 크로스 엔트로피 손실함수 계산
        loss.backward() # 손실함수 기준 역전파
        optimizer.step() # 가중치 최적화
        running_loss += loss.item()

    loss_.append(running_loss / n)
    print('[%d] loss: %.3f' %(epoch + 1, running_loss / len(trainloader)))

PATH = './cifar_net.pth' # 모델 저장 경로
torch.save(net.state_dict(), PATH) # 모델 저장

c:\users\neurophet\miniconda3\envs\myvenv\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1] loss: 1.765
[2] loss: 1.373
[3] loss: 1.219
[4] loss: 1.120
[5] loss: 1.045
[6] loss: 0.988
[7] loss: 0.933
[8] loss: 0.887
[9] loss: 0.852
[10] loss: 0.816


In [6]:
net = Net().to(device) # 모델 선언
net.load_state_dict(torch.load(PATH)) # 모델 parameter 불러오기

correct = 0
total = 0
with torch.no_grad(): # 파라미터 업데이트 같은거 안하기 때문에 no_grad를 사용.
    # net.eval() # batch normalization이나 dropout을 사용하지 않았기 때문에 사용하지 않음. 항상 주의해야함.
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1) # 10개의 class중 가장 값이 높은 것을 예측 label로 추출.
        total += labels.size(0) # test 개수
        correct += (predicted == labels).sum().item() # 예측값과 실제값이 맞으면 1 아니면 0으로 합산.

print(f'accuracy of 10000 test images: {100*correct/total}%')
print(outputs.data) # 한 epoch에서 8개의 각 배치에 대한 10개의 class에 대한 score 산출.

accuracy of 10000 test images: 63.72%
tensor([[ 2.8833,  2.9782,  4.8289,  8.3662,  4.7501,  6.3826,  5.7566,  1.7995,
          0.0000,  0.8376],
        [ 3.7793,  0.0000,  3.2714,  8.0115,  5.6060,  7.5911,  0.0000,  5.8013,
          0.0000,  1.2033],
        [ 0.0000,  1.5023,  2.9035,  8.1839,  8.5890,  5.0556,  6.0460,  1.4339,
          0.0000,  0.0000],
        [ 4.8488,  0.0000,  2.4326,  6.9934,  4.2143,  6.4438,  0.0000,  2.5219,
          4.7944,  0.0000],
        [ 0.0000,  0.0000,  6.9839,  8.0819,  4.7988,  7.6591,  8.5736,  4.6522,
          0.0000,  0.0000],
        [ 1.6088,  0.0000,  6.7264,  6.3954,  6.9381,  9.4155,  3.6968,  6.8930,
          0.0000,  0.0000],
        [ 7.9694,  5.1757,  5.8047,  4.5532,  8.2129,  3.8324,  3.6214,  2.8414,
          1.2231,  0.9064],
        [ 0.3189,  0.0000,  2.3859,  3.6186,  6.9930,  3.9335,  1.5956, 12.6324,
          0.0000,  2.0674]], device='cuda:0')
